# 뉴스기사 분류모델

## #01. 형태소 분석엔진 설치(Colab)

In [ ]:
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# !bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

## #02. 패키지 참조

In [1]:
import sys
sys.path.append('../../')
import helper

### 그 밖의 패키지 참조

In [4]:
import numpy as np
import seaborn as sb
import requests

from pandas import DataFrame, read_excel
from konlpy.tag import Mecab
from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## #03. 데이터셋 준비하기

In [3]:
origin = read_excel('./news.xlsx')
origin.head()

,category,content
0,IT_과학,"“갤럭시S9 20만 원대, 아이폰6S 0원!” 모비톡, 가정의 달 이벤트 '갤럭시노..."
1,IT_과학,'모비톡' 단독으로 진행되는 5월 가정의 달 이벤트에 이용자들의 폭발적인 반응이 나...
2,IT_과학,"이 밖에도 '모비톡'은 '갤럭시노트8', 'V30' 구매 시 '닌텐도 스위치'를 증..."
3,IT_과학,"LG 그램’, 100만대 판매기념 한정판 나왔다 LG전자가 ‘그램’ 노트북 누적판매..."
4,IT_과학,LG전자는 베스트샵 등 주요 매장에서 23일부터 그램 한정판 500대를 판매한다고 ...


### 불용어 데이터셋

In [17]:
r = requests.get("https://data.hossam.kr/korean_stopwords.txt")
r.encoding = 'utf-8'

stopwords = r.text.split("\n")


## #03. 데이터 전처리

### 한글을 제외한 영어,특수문자 제거

In [18]:
df = origin.copy()

# 한글을 제외한 나머지 글자들을 빈 문자열로 대체
df['content'] = df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)

# content 컬럼의 데이터들 중에서 빈 문자열만 존재하는 항목은 결측치로 대체
df['content'].replace('', np.nan, inplace=True)

# 전체 데이터 셋 크기 확인
print('데이터 크기: ', df['content'].shape)

# 결측치 확인
print('결측치 크기: ', df['content'].isna().sum())

데이터 크기:  (76307,)
결측치 크기:  457


### 결측치 정제

In [19]:
df.dropna(inplace=True)
df.isna().sum()

category    0
content     0
dtype: int64

### 종속변수 라벨링
#### 종속변수 값의 종류

In [20]:
category = list(df['category'].unique())
category

['IT_과학', '경제', '사회', '생활_문화', '세계', '정치']

#### 종속변수 값의 종류를 딕셔너리로 변환

In [21]:
cate_dict = {}

for i,v in enumerate(category):
    cate_dict[v] = i

cate_dict

{'IT_과학': 0, '경제': 1, '사회': 2, '생활_문화': 3, '세계': 4, '정치': 5}

#### 종속변수 라벨링

In [22]:
df['category'] = df['category'].map(cate_dict)
df['category'].unique()

array([0, 1, 2, 3, 4, 5], dtype=int64)

### 뉴스기사 형태소 분석

In [23]:
if sys.platform == 'win32':
    mecab = Mecab(dicpath="c:\mecab\mecab-ko-dic")
else:
    mecab = Mecab()

# 문장 내 형태소들을 저장할 리스트
word_set = []

# 덧글 내용에 대한 반복처리
for i,v in enumerate(df['content']):
    # 덧글 하나에 대한 형태소 분석
    morphs = mecab.morphs(v)
    print(morphs)

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/